# Introduction

<img src='../images/Screen Shot 2019-04-28 at 3.23.44 PM.png' />


# Opening the Hood

## Heroku and Amazon

Heroku's relies on AWS for its infrastructure. They rent servers from Amazon to host users' code.

<img src='../images/Screen Shot 2019-04-28 at 3.35.28 PM.png' />

**Server:** a computer that provides data to other computers. It may serve data to systems on a local area network (LAN) or a wide area network (WAN) over the Internet. Many types of servers exist, including web servers, mail servers, and file servers. Each type runs software specific to the purpose of the server.

<img src='../images/Screen Shot 2019-04-28 at 3.36.13 PM.png' />

## Linux Containers

The virtualized servers that deployment platforms provide are called linux containers. Each container has its own partitioned filesystem. Containers share server resources like memory, compute power and storage space, but not files. Heroku manages how they allocate memory across the different containers.

<img src='../images/Screen Shot 2019-04-29 at 10.47.01 AM.png' />

<img src='../images/Screen Shot 2019-04-29 at 10.47.11 AM.png' />

The main difference between a container and local environment is that you send commands to be run on a container remotely instead of interacting with it directly.

<video controls src='../images/Screen Recording 2019-04-29 at 10.48.02 AM.mov' />

# Procfile

Heroku will allow you one “dyno” or single command container for free. This means that on a free tier Heroku account, you can run one command on the container and you will not be charged for it. So many commands to choose from, which one will you pick? Well the most straightforward answer would be: the command that will run your application!

Running your application on Heroku requires explicitly telling the container or dyno to run your application. If you take a look at the Procfile in your gallery app you will see how this command is instructed to run.

<img src=https://d17h27t6h515a5.cloudfront.net/topher/2016/June/5769af4f_ud272-l2-05-01-procfile/ud272-l2-05-01-procfile.png />

The Procfile specifies a process of type “web”, which Heroku specifies to be the only process type that can accept HTTP traffic, which is necessary for your application to be properly hosted. The command that is being run on the Heroku virtual dyno is bundle exec rackup config.ru with a specified port, shown by -p and the variable $PORT

This command will automatically configure a web server with the config.ru specifications that will run on the $PORT port.

Well what are the specifications of config.ru? Open that up.

<img src=https://d17h27t6h515a5.cloudfront.net/topher/2016/June/5769afaa_ud272-l2-05-02-config-ru/ud272-l2-05-02-config-ru.png />

You can see here that config.ru loads the entire contents of the app folder which contains the gallery application. It also fires up Sinatra to run the application, as seen on line 2.

So now you can see how Heroku is able to know what to do with your code when you push it.

Heroku is able to recognize the Procfile as a file that will specify a process to run on the dyno. You can see in the terminal output of your Heroku push. It is important to reiterate the the command in the Procfile is not being run on your local environment, it is running on the production dyno where your application is hosted.

<img src=https://d17h27t6h515a5.cloudfront.net/topher/2016/June/5769afe0_ud272-l2-05-03-procfile-declares/ud272-l2-05-03-procfile-declares.png />

This is a key difference between running your app locally and hosting it on Heroku. If you were running this application locally, all you would have to do is run ruby app.rb to fire up the server and get your application running. The Procfile is specifically included so that Heroku knows what to do with the code you gave it.

# Ephemeral Filesystem

**Ephemeral Filesystem:** During the dyno's lifetime its running processes can use the filesystem as a temporary scratchpad, but no files that are written are visible to processes in any other dyno and any files written will be discarded the moment the dyno is stopped or restarted.

<img src='../images/Screen Shot 2019-04-29 at 11.15.19 AM.png' />

<img src='../images/Screen Shot 2019-04-29 at 11.15.30 AM.png' />

In [5]:
!ls ../images

Screen Recording 2019-04-29 at 10.48.02 AM.mov
Screen Shot 2019-04-27 at 8.09.40 PM.png
Screen Shot 2019-04-27 at 8.13.00 PM.png
Screen Shot 2019-04-27 at 8.20.12 PM.png
Screen Shot 2019-04-27 at 8.20.27 PM.png
Screen Shot 2019-04-28 at 12.08.14 PM.png
Screen Shot 2019-04-28 at 3.23.44 PM.png
Screen Shot 2019-04-28 at 3.35.22 PM.png
Screen Shot 2019-04-28 at 3.35.28 PM.png
Screen Shot 2019-04-28 at 3.36.13 PM.png
Screen Shot 2019-04-28 at 6.47.12 AM.png
Screen Shot 2019-04-28 at 6.47.25 AM.png
Screen Shot 2019-04-28 at 6.47.39 AM.png
Screen Shot 2019-04-28 at 6.49.33 AM.png
Screen Shot 2019-04-28 at 6.51.58 AM.png
Screen Shot 2019-04-28 at 6.52.20 AM.png
Screen Shot 2019-04-29 at 10.47.01 AM.png
Screen Shot 2019-04-29 at 10.47.11 AM.png
Screen Shot 2019-04-29 at 10.47.22 AM.png
Screen Shot 2019-04-29 at 11.15.19 AM.png
Screen Shot 2019-04-29 at 11.15.30 AM.png


# Setting Up a Database

The repository you cloned in Lesson 1 contained multiple branches that represent different stages of the application. For this step you will need to create a new local branch within your repository and set it to be track the remote “pg” branch. To do this make sure that you are in your repo and run `git checkout -b pg origin/pg`

<img src=https://d17h27t6h515a5.cloudfront.net/topher/2016/June/5769b669_ud272-l2-07-02-checkout-pg/ud272-l2-07-02-checkout-pg.png />

We named this branch is called “pg” because the database that we have is a PostgreSQL database. There are many databases out there that you can use but Heroku requires that you use PostgreSQL when using a Ruby application. Sinatra doesn’t require any database by default but other frameworks like Rails come with SQLite as a default. SQLite will create a database file in your application directory that will change as your database updates. Because of Heroku’s ephemeral filesystem this changing database file will be eventually reset, meaning it is unsuitable for use with Heroku.

You may notice when opening this up in your editor that there are many more files present than there were in the version without a database. This is because to get a database working with this Sinatra application you have to have not only the database but also a way for your application to interact with the database, in this case ActiveRecord. ActiveRecord is an ORM or Object Relational Manager that will translate the ruby code to SQL (pronounced sequel) that the database can execute.

<img src=https://d17h27t6h515a5.cloudfront.net/topher/2016/June/5769b6d6_ud272-l2-07-03-pg-tree/ud272-l2-07-03-pg-tree.png />

ActiveRecord comes in the form of a gem so you want to make sure that the application is stable locally before it can be deployed online, so run `bundle install`

<img src=https://d17h27t6h515a5.cloudfront.net/topher/2016/June/5769b6fb_ud272-l2-07-04-bundle-install/ud272-l2-07-04-bundle-install.png />

Now that the required gems are installed on your local system, open the app as you regularly would on your local development environment. Run `ruby app.rb` When you try to open the application by visiting localhost:4567 you see that we have a Sinatra error:

<img src=https://d17h27t6h515a5.cloudfront.net/topher/2016/June/5769cb63_ud272-l2-07-05-undefined-pg/ud272-l2-07-05-undefined-pg.png />

Here we see that the table that needs to be in the database does not yet exist. You need to create it. Return to the command line, shut the server down and run `rake db:migrate`

<img src=https://d17h27t6h515a5.cloudfront.net/topher/2016/June/5769cbe9_ud272-l2-07-06-db-migrate/ud272-l2-07-06-db-migrate.png />

Now that you have created the table required, go ahead and fire up the server again with `ruby app.rb` and navigate to it on your browser.

<img src=https://d17h27t6h515a5.cloudfront.net/topher/2016/June/5769cc11_ud272-l2-07-07-local-homepage/ud272-l2-07-07-local-homepage.png />

Now that the application is running without errors you are ready to deploy this new version of the application to Heroku.

To push your pg branch to the Heroku remote make sure that you are on the pg branch (you can check by running git branch) and run this command: `git push heroku HEAD:master`

<img src=https://d17h27t6h515a5.cloudfront.net/topher/2016/June/5769cc4d_ud272-l2-07-08-push-pg/ud272-l2-07-08-push-pg.png />

This command will ensure that the code that is pushed to heroku master is the one that is at HEAD. In this case, this is the pg branch.

Once you do this open the application by running heroku open.

<img src=https://d17h27t6h515a5.cloudfront.net/topher/2016/June/5769cc71_ud272-l2-07-09-internal-server-error/ud272-l2-07-09-internal-server-error.png />

You will see that we get an internal server error. Well if you recall, when running the application locally you had to run the database migrations. To run a command on your Heroku server you have to prepend the command with heroku run. This will tell Heroku that you want to run a command on the live server, much like you would when you are running it on localhost. So instead of `rake db:migrate`, try `heroku run rake db:migrate`.

<img src=https://d17h27t6h515a5.cloudfront.net/topher/2016/June/5769ccf1_ud272-l2-07-10-no-addon/ud272-l2-07-10-no-addon.png />

Well that’s strange, it looks different than the different error you got last time about the undefined PG table. It looks here that Heroku is not able to find a database connection.

This happened because you do not have a PostgreSQL addon included in Heroku. Heroku will only automatically include the PostgreSQL addon if your very first deploy includes the gem pg in the Gemfile. This is your second deploy to the same Heroku application so it is not included. To include this addon run heroku addons:create heroku-postgresql.

<img src=https://d17h27t6h515a5.cloudfront.net/topher/2016/June/5769cd1a_ud272-l2-07-11-addon-create/ud272-l2-07-11-addon-create.png />

Now try running the database migrations again in the Heroku server by using `heroku run rake db:migrate`.

<img src=https://d17h27t6h515a5.cloudfront.net/topher/2016/June/5769cd3b_ud272-l2-07-12-db-migrate-working/ud272-l2-07-12-db-migrate-working.png />

Great it looks like you’re almost there. Check if your page is up by refreshing it.

<img src=https://d17h27t6h515a5.cloudfront.net/topher/2016/June/5769cd63_ud272-l2-07-13-deployed-live/ud272-l2-07-13-deployed-live.png />

You have now successfully deployed the version of the Udacity Gallery application with a database!

Let’s recap the takeaways here:

- Running commands on your Heroku server is different than running those same commands on your local computer.
    - Prepend your commands with `heroku run`.
- Installing a database on your Heroku server is not always automatic.
    - If your first deploy contains the pg gem, then Heroku will include the `heroku-postgresql` addon.
    - If your application does not start with a postgres database but you want to add it you must run the `heroku addons:create heroku-postgresql`.

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />

# 

<img src='../images/' />

<img src='../images/' />